In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np 
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import concatenate
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import keras
import tensorflow as tf
import scipy.stats as stats
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
from keras.layers import  Reshape
from keras.layers import Conv2D, MaxPooling2D
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4289755599607155714
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7235272160524757301
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15870492672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16929678851612681462
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10745395968577157780
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
os.chdir('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/')

# %% [code]

# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath,sep = ',', header=None)
    return dataframe.to_numpy()
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = concatenate(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(sample='', prefix='',sensor='',recorder=''):
#     sensor = 'accel'
#     recorder = 'phone'
    filepath = recorder +'/'+ sensor +'/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
   
    filenames += ['data_'+sample+'_'+sensor+'_'+recorder+'.txt']
            
    for filename in filenames:
        df = load_file(filepath+filename)
    return df
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    prefix = ''
    dataSampleTrain = list()
    recorder = ['phone','watch']
    sensor  = ['accel','gyro']
    for i in range (1600,1620):
        size = 99999
        data = list()
        for rec in recorder:
            for sens in sensor:
                DS = load_dataset_group(str(i), prefix,sens,rec)
                size = min(size,DS.shape[0])
                data.append(DS)
        
        
            
        for i in range(4):
            data[i] = data[i][:size]
        dataSampleTrain.append(concatenate(data, axis = 1))
        
    
    dataSampleTrain = concatenate(dataSampleTrain)
    Traindf = pd.DataFrame(dataSampleTrain)
    #Traindf.to_csv('TrainDF.csv')
    
    dataSampleTest = list() 
    for i in range (1620,1634):
        size = 99999
        data = list()
        for rec in recorder:
            for sens in sensor:
                DS = load_dataset_group(str(i), prefix,sens,rec)
                size = min(size,DS.shape[0])
                data.append(DS)
        
        
            
        for i in range(4):
            data[i] = data[i][:size]
        dataSampleTest.append(concatenate(data, axis = 1))
        
    
    dataSampleTest = concatenate(dataSampleTest)
    Testdf = pd.DataFrame(dataSampleTest)
    #Traindf.to_csv('TrainDF.csv')
    
    return Traindf,Testdf
trainDS,testDS = load_dataset('')
trainDS.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1600,A,252207666810782,-0.364761,8.7935,1.0550842;,1600,A,252207918580802,-0.85321,...,90426708196641,7.09162,-0.591667,8.195502;,1600,A,90426757696641,0.314944,-1.02228,-0.3099616;
1,1600,A,252207717164786,-0.87973,9.76878,1.0169983;,1600,A,252207968934806,-0.875137,...,90426757696641,4.97276,-0.158317,6.6967316;,1600,A,90426807196641,0.387382,-0.618541,-0.048971802;
2,1600,A,252207767518790,2.0015,11.1091,2.619156;,1600,A,252208019288809,-0.720169,...,90426807196641,3.25372,-0.191835,6.107758;,1600,A,90426856696641,0.0709985,-0.20948,-0.1959783;
3,1600,A,252207817872794,0.450623,12.6516,0.18455505;,1600,A,252208069642813,-0.57164,...,90426856696641,2.80122,-0.155922,5.997625;,1600,A,90426906196641,0.0379753,0.254976,-0.1565635;
4,1600,A,252207868226798,-2.16435,13.9284,-4.4224854;,1600,A,252208119996817,-0.380493,...,90426906196641,3.77087,-1.05135,7.731027;,1600,A,90426955696641,0.0731291,0.719431,-0.0010349044;


In [4]:
# Preprocess and load X
def X_pre(trainDS):
    
    
    cols = [5,11,17,23]
    for col in cols:
            trainDS[trainDS.columns[col]] = trainDS[trainDS.columns[col]].str.replace(';','').astype(float)
    
    columns = ['x','y','z']
    X = pd.DataFrame(data=None, columns=columns)
    X['x'] = trainDS[3]+trainDS[9]+trainDS[15]+trainDS[21]
    X['y'] = trainDS[4]+trainDS[10]+trainDS[16]+trainDS[22]
    X['z'] = trainDS[5]+trainDS[11]+trainDS[17]+trainDS[23]
    
    # train the normalization
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X

trainX = X_pre(trainDS)
testX = X_pre(testDS)

In [5]:
# Preprocess and load Y

def Y_pre(trainDS):
    cols = [1,7,13,19]
    Y = np.zeros((trainDS.shape[0]),dtype=int)

    lable_encoder = LabelEncoder()

    pos = list()
    for col in cols:
        y = trainDS[col]
        y = lable_encoder.fit_transform(y)
        pos.append(y)

    pos = np.vstack((pos))

    for i in range (trainDS.shape[0]):
        Y[i] = np.bincount(pos[:,i]).argmax()

    return Y

trainY = Y_pre(trainDS)
testY = Y_pre(testDS)

In [6]:
Fs=20
frame_size = Fs*4 #80
hop_size = Fs*2 #40

In [7]:
def get_frames(df,tag, frame_size, hop_size):
    
    N_FEATURES = df.shape[1]
    frames = []
    labels = []
    for i in range(0,len(df )- frame_size, hop_size):
        x1 = df[i: i+frame_size,0]
        y1 = df[i: i+frame_size,1]
        z1 = df[i: i+frame_size,2]
        
        label = stats.mode(tag[i: i+frame_size])[0][0]
        frames.append([x1,y1,z1])
        labels.append(label)
        
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [8]:
with tf.device('/device:GPU:0'):
    train_X,train_Y = get_frames(trainX,trainY, frame_size, hop_size)
    test_X,test_Y = get_frames(testX,testY, frame_size, hop_size)
    trainx = train_X.reshape(train_X.shape[0], 80, 3,1)
    testx = test_X.reshape(test_X.shape[0], 80, 3,1)

In [11]:

x_train,y_train,x_test,y_test = trainx,train_Y,testx,test_Y
with tf.device('/device:GPU:0'):
    #batch_size = 12
    model = Sequential()
    model.add(Conv2D(256, (2,2), activation = 'relu', input_shape = x_train[0].shape))
    #model.add(Dropout(0.2))

    model.add(Conv2D(512, (2,2), activation = 'relu'))
    #model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(1024, activation = 'relu'))
    #model.add(Dropout(0.5))

    model.add(Dense(18, activation='softmax'))
    
    model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 
    history = model.fit(x_train, y_train, epochs = 25, validation_data=(x_test, y_test), verbose=1 )
    

Train on 16096 samples, validate on 23638 samples
Epoch 1/25
16096/16096 [==============================] - 17s 1ms/step - loss: 1.7689 - accuracy: 0.4026 - val_loss: 3.1394 - val_accuracy: 0.2249
Epoch 2/25
16096/16096 [==============================] - 13s 803us/step - loss: 1.0713 - accuracy: 0.6323 - val_loss: 3.4448 - val_accuracy: 0.2283
Epoch 3/25
16096/16096 [==============================] - 13s 805us/step - loss: 0.7936 - accuracy: 0.7327 - val_loss: 3.9186 - val_accuracy: 0.2284
Epoch 4/25
 2144/16096 [==>...........................] - ETA: 8s - loss: 0.6130 - accuracy: 0.7836

KeyboardInterrupt: 